In [2]:
import seisbench
import seisbench.data as sbd
import seisbench.generate as sbg
import seisbench.models as sbm
from seisbench.util import worker_seeding

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import pandas as pd
import datetime
import h5py

In [3]:

dfS = pd.read_parquet(
    "https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_stations.parquet"
)
dfC = pd.read_parquet(
    "https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_fullcatalog.parquet")
dfC['pick_time']=pd.to_datetime(dfC['pick_time'],format='%Y-%m-%dT%H:%M:%S.%fZ',errors='coerce')

In [6]:
# Break it down to just a day!
t1 = datetime.datetime(2019,1,1)
t2 = datetime.datetime(2019,1,1,10)
dfC = dfC[(dfC['pick_time'] > t1) & (dfC['pick_time'] < t2)]

# Let's just grab P-waves for now- can grab S later
dfC = dfC[(dfC['phase'] == 'P')]

# Also only use picks on stations that are in our station list!
# ground_truth = ground_truth[([t in df['id'].to_list() for t in ground_truth['sta_code']])]

In [7]:
dfC.head()

,phase,azimuth,distance,time_residual,time_weight,pick_time,network,station,channel,pick_id,...,ev_longitude,ev_latitude,ev_depth,ev_magnitude,mag_type,ev_phasecount,sta_lon,sta_lat,sta_elev,sta_unit
0,P,6.76,0.411,-0.014,0.265,2019-01-01 05:06:05.601,XO,LT20,HHZ,quakeml:earthquake.alaska.edu/pick/95,...,-161.4584,54.3925,25600.0,2.4,ml,35,-161.374600,54.800200,-150.0,m/s
1,P,76.00,0.751,0.343,0.265,2019-01-01 05:06:10.941,XO,LT17,HHZ,quakeml:earthquake.alaska.edu/pick/97,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.201600,54.567700,-114.0,m/s
2,P,106.40,0.733,1.011,0.265,2019-01-01 05:06:11.047,XO,WS75,HHZ,quakeml:earthquake.alaska.edu/pick/98,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.256908,54.179630,-1108.9,m/s
3,P,132.56,0.895,-1.155,0.265,2019-01-01 05:06:11.579,XO,WD69,HHZ,quakeml:earthquake.alaska.edu/pick/99,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.342890,53.782065,-3895.9,m/s
5,P,106.85,0.965,0.411,0.265,2019-01-01 05:06:13.566,XO,LD45,HHZ,quakeml:earthquake.alaska.edu/pick/101,...,-161.4584,54.3925,25600.0,2.4,ml,35,-159.882900,54.102500,-2019.0,m/s


### Write formats to data_format group

In [38]:
# Pull in example data_format group from the seisbench dummy dataset:
import h5py
filename = "/home/jovyan/.seisbench/datasets/dummydataset/waveforms.hdf5"
f = h5py.File(filename, 'r')
g_example = f['data_format']

In [42]:
print(g_example.attrs)

<Attributes of HDF5 object at 140231852476352>


In [ ]:
create_dataset_like(name, other, **kwds)

### Write waveforms to data group

In [34]:
client = Client("iris")

with h5py.File('test_data5.h5', 'w') as hf:
    stacked = []
    for i in range(5):
        pick = dfC.iloc[i]

        t1 = UTCDateTime(pick['pick_time'] - pd.Timedelta(minutes=2))
        t2 = UTCDateTime(pick['pick_time'] + pd.Timedelta(minutes=12))

        wf = client.get_waveforms(pick['network'],pick['station'],'  ',pick['channel'][0:2]+'*',t1,t2)

        wf_np = np.stack([t.data for t in wf])
        
        stacked.append(wf_np)
    
    trace_len = np.shape(stacked)[2]
    g = hf.create_group('data')    
    g.create_dataset('bucket0',shape=[5,3,trace_len],dtype='f8',data=stacked)

In [36]:
# Round trip

filename = "/home/jovyan/ak-retraining/retrain_data/test_data5.h5"

f = h5py.File(filename, 'r')

print(list(f.keys()))

dset = f['data']
for name in dset:
    print(dset[name])

['data']
<HDF5 dataset "bucket0": shape (5, 3, 84000), type "<f8">


In [37]:
tr = f['data']['bucket0'][0,:3,:trace_len]
print(tr)

[[ 22505.  22349.  22178. ...,  -7004.  -7384.  -7740.]
 [ 12314.  11493.  10668. ...,  14477.  14232.  13965.]
 [-22583. -22252. -21931. ..., -28755. -28713. -28643.]]
